# Homework 1 - Streaming (15 pts)

Please turn in your note book naming as **BDM\_HW1\_NetID.ipynb**.


## Task 1 (5 points)

You are asked to write a Python generator that takes a sequence of characters as input (think of it as a string iterator that can only be traversed forward) and returns a generator of substrings containing consecutive characters in the string. For example, substrings of consecutive characters in `'Good morning !!!'` are `'oo'` and `'!!!'`. Your task is to complete the generator below (only fill in the body of **`findDuplicateSequence()`**).

In [1]:
def findDuplicateSequence(s):
    # YOU CAN ONLY EDIT BELOW THIS LINE
    i = next(s, 'end')
    if i != 'end':
      j = next(s, 'end')
      if j != 'end':  
        temp = i
        while (True):
          if j == i:  temp += j
          else:
            if len(temp) > 1: yield temp
            i, temp = j, j
          j = next(s, 'end')
          if j == 'end':
            if len(temp) > 1: yield temp
            break

In [2]:
def findDuplicateSequence(s):
    from itertools import groupby
    yield from filter(lambda y: len(y)>1, map(lambda x: ''.join(x[1]), groupby(s)))

In [3]:
# DO NOT EDIT THIS CELL, YOU CAN USE THIS TO CHECK THE CODE IN THE PREVIOUS CELL

print(repr(next(findDuplicateSequence(iter('aabccccedaaa'))))) # 'aa'
print(list(findDuplicateSequence(iter('aabccccedaaa')))) # 'aa', 'cccc', 'aaa'
print(list(findDuplicateSequence(iter('Good morning!!!')))) # 'oo', '!!!'

'aa'
['aa', 'cccc', 'aaa']
['oo', '!!!']



## Task 2 (10 points)

We are greatly inspired by the [Consumer Complaints](https://github.com/InsightDataScience/consumer_complaints) challenge from [InsightDataScience](https://github.com/InsightDataScience/). In fact, we are going to tackle the same challenge but streaming. Please read through the challenge at the following link (the most important sections are “Input dataset” and “Expected output”):

<https://github.com/InsightDataScience/consumer_complaints>

Our problem is different than the one above in that the INPUT is also <ins>sorted by product (alphabetically) and year (ascending)</ins>, similar to the requested output format. You are asked to complete the generator below to produce the expected output. You should not edit any other code.

### INPUT
Your code will be evaluated against the original data set (in CSV format) downloaded from the below link (of course with a sorted version). For your convenience, a smaller version of the file is provided below for testing purposes.

https://www.consumerfinance.gov/data-research/consumer-complaints/#download-the-data



In [4]:
# !gdown --id 1-IeoZDwT5wQzBUpsaS5B6vTaP-2ZBkam -O complaints.csv
# !head -n 1 complaints.csv

In [5]:
def task2(reader):
  def mapper(_, reader):
    yield ((reader['Product'].lower(), reader['Date received'].split('-')[0]), reader['Company'].lower())

  def reducer(record, counts):
    yield (str(record[0]), str(record[1]), len(counts), len(set(counts)), round(100 * max([counts.count(i) / len(counts) for i in set(counts)])))

  yield list(mr.run(enumerate(reader), mapper, reducer))

In [6]:
# DO NOT EDIT THIS CELL, YOU CAN RUN TO QUICKLY CHECK YOUR RESULTS

import csv
import mapreduce as mr
with open('complaints.csv', 'r') as fi, open('output.csv', 'w') as fo:
  reader = csv.DictReader(fi)
  writer = csv.writer(fo)
  for row in task2(reader):
    writer.writerow(row)

!head output.csv

In [7]:
#Another version with mapreduce method, but not generator

# def returnInfo(reader):
#   time = reader['Date received'].split('-')
#   return (reader['Product'].lower(), reader['Company'].lower(), time[0])

# def union_companies_for_product_and_year(result, record):
#   product, company, year = record[0], record[1], record[2]
#   result[(product, year)] = result.get((product, year), []) + [company]
#   return result

# def combine_result_func(result, record):
#   product, year, companies = record[0], record[1], record[2]
#   highest_percent = 0
#   for company in set(companies):
#     highest_percent = max(highest_percent, companies.count(company) / len(companies))
#   result[(product, year)] = result.get((product, year), []) + [len(companies), len(set(companies)), round(100 * highest_percent)]
#   return result

# def output_format(record):
#   return (record[0], record[1], record[2][0], record[2][1], record[2][2])